## Scraping Multiple Thread Tribunews

In [9]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading
from urllib.request import Request, urlopen

In [10]:
jumlah_index = 1
threads_link = []
links = []

In [11]:
def scrape_links(page_number):
    url = f"https://www.tribunnews.com/2023/09?page={page_number}"
    try:
        response = requests.get(url, headers={'User-Agent': 'Chrome/117.0.0.0'})
        response.raise_for_status()  # Raise an exception for HTTP errors
        soup = BeautifulSoup(response.text, 'html.parser')
        articles = soup.find_all('li', {"class": "ptb15"})
        page_links = []
        for article in articles:
            header = article.find('h3', {"class": "fbo f16"})
            link = header.find('a')['href']
            links.append(link)
        print(f"Scraped {len(page_links)} links from page {page_number}")
        return page_links
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching data from {url}: {e}")   

In [12]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 0 links from page 19
Scraped 0 links from page 1
Scraped 0 links from page 5
Scraped 0 links from page 14
Scraped 0 links from page 3
Scraped 0 links from page 20
Scraped 0 links from page 21
Scraped 0 links from page 16
Scraped 0 links from page 29
Scraped 0 links from page 2
Scraped 0 links from page 12
Scraped 0 links from page 4
Scraped 0 links from page 32
Scraped 0 links from page 17
Scraped 0 links from page 9
Scraped 0 links from page 35
Scraped 0 links from page 13
Scraped 0 links from page 15
Scraped 0 links from page 44
Scraped 0 links from page 18
Scraped 0 links from page 6
Scraped 0 links from page 34
Scraped 0 links from page 43
Scraped 0 links from page 26
Scraped 0 links from page 8
Scraped 0 links from page 11
Scraped 0 links from page 46
Scraped 0 links from page 37
Scraped 0 links from page 27
Scraped 0 links from page 7
Scraped 0 links from page 42
Scraped 0 links from page 39
Scraped 0 links from page 23
Scraped 0 links from page 55
Scraped 0 links from pa

In [13]:
results = []
def scrape_url(url):
    try:
        response = requests.get(url, headers={'User-Agent': 'Chrome/117.0.0.0'})
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Judul Berita
            title_elem = soup.find('h1', {"id": "arttitle"})
            if title_elem:
                title_text = title_elem.text.strip()
            else:
                title_text = "Title not found"
            # Author berita
            author_elem = soup.find('div', {"id": "penulis"})
            if author_elem:
                penulis = author_elem.find('a')
                author_text =penulis.text.strip()
            else:
                author_text = "Author not found"
               # Editor Berita
            editor_elem = soup.find('div', {"id": "editor"})
            
            if editor_elem:
                editors = editor_elem.find('a')
                editor_text = editors.text.strip()
            else:
                editor_text = "editor not found" 
            
            # tanggal berita
            date_elem = soup.find('time', {"class": "grey"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            category_elements = soup.find_all('span',{"itemprop": "name"})
            if category_elements:
                category_text= category_elements[2].get_text()
            else:
                category_text = "Category not found"
            ## Content Berita
            body_elem = soup.find('div', {"class": "side-article txt-article multi-fontsize"})
            
            if body_elem:
                content_elem = body_elem.find_all('p')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
              content_text ="Content not found"

            results.append({'title': title_text,
                            'author' : author_text,
                            'editor' : editor_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [14]:
threads = []
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

Error processing URL 'https://www.tribunnews.com/australia-plus/2023/09/25/dunia-hari-ini-sampel-asteroid-berusia-miliaran-tahun-sudah-sampai-di-bumi': list index out of range
Error processing URL 'https://www.tribunnews.com/kesehatan/2023/09/26/sering-muncul-tanpa-gejala-ini-waktu-yang-dianjurkan-untuk-deteksi-dini-hindari-kanker-prostat': list index out of rangeError processing URL 'https://www.tribunnews.com/superskor/2023/09/23/timnas-indonesia-hadapi-laga-penentuan-vs-korea-utara-robi-darwis-kami-jauh-lebih-siap': list index out of range

Error processing URL 'https://www.tribunnews.com/pendidikan/2023/09/25/kunci-jawaban-tema-3-kelas-6-halaman-111-pembelajaran-6-subtema-2-buku-tematik': list index out of range
Error processing URL 'https://www.tribunnews.com/pendidikan/2023/09/25/kunci-jawaban-tema-3-kelas-6-halaman-115-dan-116-buku-tematik-apa-topik-masalah-teks-tersebut': list index out of range
Error processing URL 'https://www.tribunnews.com/bisnis/2023/09/25/sukses-lakukan-t

In [15]:
df = pd.DataFrame(results)

df.head(10)

,title,author,editor,category,date,content,link
0,Buntut Aksi Anarkis Pengunjuk Rasa di Pohuwato...,Author not found,Erik S,Sulawesi,"Senin, 25 September 2023 15:54 WIB","TRIBUNNEWS.COM, GORONTALO- Polisi menetapkan l...",https://www.tribunnews.com/regional/2023/09/25...
1,Formasi PPPK Kemenperin 2023 untuk Lulusan SMK...,Widya Lisfianti,bunga pradipta p,Umum,"Senin, 25 September 2023 15:52 WIB",TRIBUNNEWS.COM - Kementerian Perindustrian (Ke...,https://www.tribunnews.com/nasional/2023/09/25...
2,Calon Hakim MK Elita Rahmi Usul Sistem Pengawa...,Chaerul Umam,Malvyandie Haryadi,Hukum,"Senin, 25 September 2023 15:43 WIB","Laporan Wartawan Tribunnews.com, Chaerul Umam\...",https://www.tribunnews.com/nasional/2023/09/25...
3,"Tak Ada Tanda Apa pun Sebelum Meninggal, Latie...",Rinanda DwiYuliawati,Salma Fenty,Gosipi,"Selasa, 26 September 2023 08:29 WIB",TRIBUNNEWS.COM - Tak ada tanda apa pun sebelum...,https://www.tribunnews.com/seleb/2023/09/26/ta...
4,Lowongan Dosen CPNS 2023 di Sejumlah Kementeri...,Pondra Puger Tetuko,Daryono,Umum,"Senin, 25 September 2023 15:51 WIB",TRIBUNNEWS.COM - Simak lowongan dosen CPNS 202...,https://www.tribunnews.com/nasional/2023/09/25...
5,Gaji PPPK Kementerian PUPR dan Link Download PDF,Widya Lisfianti,Garudea Prabawati,Umum,"Selasa, 26 September 2023 08:18 WIB",TRIBUNNEWS.COM - Kementerian Pekerjaan Umum da...,https://www.tribunnews.com/nasional/2023/09/26...
6,Prabowo Beri Penghargaan Dharma Pertahanan ke ...,Reza Deni,Adi Suhendi,Hukum,"Senin, 25 September 2023 15:46 WIB","Laporan Reporter Tribunnews.com, Reza Deni\nTR...",https://www.tribunnews.com/nasional/2023/09/25...
7,"Usai Rapat di Istana, Pemerintah Akan Terbitka...",Taufik Ismail,Adi Suhendi,Hukum,"Senin, 25 September 2023 15:39 WIB",Laporan Wartawan Tribunnews Taufik Ismail\nTRI...,https://www.tribunnews.com/nasional/2023/09/25...
8,"Sepakat dengan Menkominfo Budi Arie, ADKI Mint...",Wahyu Aji,Seno Tri Sulistiyono,News,"Senin, 25 September 2023 15:42 WIB","TRIBUNNEWS.COM, JAKARTA - Menteri Komunikasi d...",https://www.tribunnews.com/techno/2023/09/25/s...
9,"Saling Sindir soal Kotoran Udang, Codeblu Bala...",Ayu Miftakhul,bunga pradipta p,Gosipi,"Senin, 25 September 2023 15:37 WIB",TRIBUNNEWS.COM - Aksi saling sindir dua food v...,https://www.tribunnews.com/seleb/2023/09/25/sa...


save_date

In [16]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
excel_file_name = f'./excel/tribunnews_{current_datetime}.xlsx'
df.to_excel(excel_file_name, index=False)

print(f'Data has been saved to {excel_file_name}')

Data has been saved to ./excel/tribunnews_2023-09-26_08-37-27.xlsx
